In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('merged_datasets_cafeteria_hansard_foodon_snomedct.csv', index_col=0)

/opt/conda/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [82]:
df['clean_labels']=df['labels'].apply(lambda x: x.replace('B-','').replace('I-',''))

In [42]:
hansard_df = df.query("dataset=='hansard'")

In [37]:
hansard_df = hansard_df[hansard_df['labels'].str.contains("B-")]

In [45]:
hansard_df['clean_labels']=hansard_df['labels'].apply(lambda x: x.replace('B-','').replace('I-',''))

/tmp/ipykernel_2298/2838982477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hansard_df['clean_labels']=hansard_df['labels'].apply(lambda x: x.replace('B-','').replace('I-',''))


In [70]:
bad_classes_of_interest=['AG.01.ae','AG.01.d.04','AG.01.h.02.b','AG.01.n.03','AG.01.n.06','AG.01.n.07','AG.01.n.08','AG.01.n.12','AG.01.n.16']
hansard_bad=hansard_df.query("clean_labels in @bad_classes_of_interest")
hansard_bad.groupby('clean_labels').count()

,words,predictions,sentence_id,word_id,labels,full sentence,fold,model_name,dataset
clean_labels,,,,,,,,,
AG.01.ae,20,20,20,20,20,20,20,20,20
AG.01.d.04,4,4,4,4,4,4,4,4,4
AG.01.h.02.b,72,72,72,72,72,72,72,72,72
AG.01.n.03,4,4,4,4,4,4,4,4,4
AG.01.n.06,124,124,124,124,124,124,124,124,124
AG.01.n.07,68,68,68,68,68,68,68,68,68
AG.01.n.08,176,176,176,176,176,176,176,176,176
AG.01.n.12,20,20,20,20,20,20,20,20,20
AG.01.n.16,20,20,20,20,20,20,20,20,20


In [93]:
def show_phrases_per_class(classes_of_interest):
    mappings={}
    for class_of_interest in classes_of_interest:
        classes_of_interest=[class_of_interest]
        hansard_interesting=df.query("clean_labels in @classes_of_interest")
        hansard_interesting['words']=hansard_interesting['words'].apply(lambda x: x.lower())
        phrases = []
        last_word_id=-5
        for index, row in hansard_interesting.iterrows():
            if row['word_id']==last_word_id+1 and row['labels'].startswith('I-'):
                phrases[-1]+= ' ' + row['words']
                last_word_id=row['word_id']
            else:
                phrases.append(row['words'])
                last_word_id=row['word_id']
        print(class_of_interest)
        value_counts=pd.DataFrame(phrases).value_counts()
        print(value_counts)
        mappings[class_of_interest]=value_counts.index[0][0]
    return mappings

In [84]:
show_phrases_per_class(good_classes_of_interest)

/tmp/ipykernel_2298/2075173569.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hansard_interesting['words']=hansard_interesting['words'].apply(lambda x: x.lower())


AG.01.y.01.b
beer                   212
gluten - free beer      12
gluten - free beers      8
pilsner beer             8
bamboo beer              4
beers                    4
dtype: int64
AG.01.n.17.b
ice cream              36
chocolate ice cream    16
ice                     4
ice - cream             4
dtype: int64
AG.01.n.14
cookies                  40
biscuits                 16
chocolate bar cookies     8
fat - free cookies        8
sweet biscuits            8
bamboo cookies            4
high - fat cookies        4
wheatflake biscuits       4
dtype: int64
AG.01.n.02
miso                   32
soups                  28
miso soup              16
soup                   12
milk - based soup       4
milk - based soups      4
palm oil based soup     4
porridge                4
stock                   4
vegetable soup          4
dtype: int64
AG.01.h.01.d
pears    20
pear      8
dtype: int64
AG.01.h.01.a
orange           40
citrus fruits    36
citrus fruit     28
citrus           16
oranges

In [85]:
show_phrases_per_class(bad_classes_of_interest)

AG.01.ae
malt             12
malted grains     4
dtype: int64


/tmp/ipykernel_2298/2075173569.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hansard_interesting['words']=hansard_interesting['words'].apply(lambda x: x.lower())


AG.01.d.04
lamb    4
dtype: int64
AG.01.h.02.b
celery          52
stick celery     8
celeriac         4
dtype: int64
AG.01.n.03
puddings    4
dtype: int64
AG.01.n.06
bolognese sauce                16
bake - only chicken nuggets     4
bake - only nuggets             4
barbecued meat                  4
chicken nuggets                 4
grilled beef                    4
hamburgers                      4
nuggets                         4
paté or high - fat meat         4
dtype: int64
AG.01.n.07
canned fish                         4
dish of fish                        4
preserved fish                      4
salted / dried fish                 4
salted / fermented fish products    4
tsukudani                           4
dtype: int64
AG.01.n.08
dried plums          28
raisins              24
dried fruit           8
apple chips           4
applesauce            4
dried grapes          4
dried plum            4
dried plum powder     4
dried raisins         4
dried vine fruits     4
fruit produc

In [65]:
good_classes_of_interest=['AG.01.y.01.b', 'AG.01.n.17.b', 'AG.01.n.14', 'AG.01.n.02', 'AG.01.h.01.d', 'AG.01.h.01.a', 'AG.01.d.05 - Pork']
hansard_df.query("clean_labels in @good_classes_of_interest").groupby('clean_labels').count()

,words,predictions,sentence_id,word_id,labels,full sentence,fold,model_name,dataset
clean_labels,,,,,,,,,
AG.01.h.01.a,208,208,208,208,208,208,208,208,208
AG.01.h.01.d,28,28,28,28,28,28,28,28,28
AG.01.n.02,168,168,168,168,168,168,168,168,168
AG.01.n.14,160,160,160,160,160,160,160,160,160
AG.01.n.17.b,136,136,136,136,136,136,136,136,136
AG.01.y.01.b,320,320,320,320,320,320,320,320,320


In [78]:
snomedct_tags={'http://purl.bioontology.org/ontology/SNOMEDCT/102260001;http://purl.bioontology.org/ontology/SNOMEDCT/762952008;http://purl.bioontology.org/ontology/SNOMEDCT/226888007',
 'http://purl.bioontology.org/ontology/SNOMEDCT/102261002',
 'http://purl.bioontology.org/ontology/SNOMEDCT/102261002;http://purl.bioontology.org/ontology/SNOMEDCT/227418000;http://purl.bioontology.org/ontology/SNOMEDCT/735051003',
 'http://purl.bioontology.org/ontology/SNOMEDCT/102261002;http://purl.bioontology.org/ontology/SNOMEDCT/227601008',
 'http://purl.bioontology.org/ontology/SNOMEDCT/102262009',
 'http://purl.bioontology.org/ontology/SNOMEDCT/102262009;http://purl.bioontology.org/ontology/SNOMEDCT/227708009;http://purl.bioontology.org/ontology/SNOMEDCT/255621006',
 'http://purl.bioontology.org/ontology/SNOMEDCT/102262009;http://purl.bioontology.org/ontology/SNOMEDCT/230118003',
 'http://purl.bioontology.org/ontology/SNOMEDCT/129560007;http://purl.bioontology.org/ontology/SNOMEDCT/72511004;http://purl.bioontology.org/ontology/SNOMEDCT/129559002',
 'http://purl.bioontology.org/ontology/SNOMEDCT/129561006;http://purl.bioontology.org/ontology/SNOMEDCT/22836000;http://purl.bioontology.org/ontology/SNOMEDCT/129559002',
 'http://purl.bioontology.org/ontology/SNOMEDCT/13577000',
 'http://purl.bioontology.org/ontology/SNOMEDCT/13577000;http://purl.bioontology.org/ontology/SNOMEDCT/102260001;http://purl.bioontology.org/ontology/SNOMEDCT/762952008;http://purl.bioontology.org/ontology/SNOMEDCT/226888007',
 'http://purl.bioontology.org/ontology/SNOMEDCT/13577000;http://purl.bioontology.org/ontology/SNOMEDCT/70813002',
 'http://purl.bioontology.org/ontology/SNOMEDCT/226041007',
 'http://purl.bioontology.org/ontology/SNOMEDCT/226064009',
 'http://purl.bioontology.org/ontology/SNOMEDCT/226465004',
 'http://purl.bioontology.org/ontology/SNOMEDCT/226470006;http://purl.bioontology.org/ontology/SNOMEDCT/29263009;http://purl.bioontology.org/ontology/SNOMEDCT/70813002',
 'http://purl.bioontology.org/ontology/SNOMEDCT/226474002;http://purl.bioontology.org/ontology/SNOMEDCT/16313001;http://purl.bioontology.org/ontology/SNOMEDCT/70813002',
 'http://purl.bioontology.org/ontology/SNOMEDCT/226888007',
 'http://purl.bioontology.org/ontology/SNOMEDCT/227411006',
 'http://purl.bioontology.org/ontology/SNOMEDCT/227601008',
 'http://purl.bioontology.org/ontology/SNOMEDCT/227708009;http://purl.bioontology.org/ontology/SNOMEDCT/255621006',
 'http://purl.bioontology.org/ontology/SNOMEDCT/22836000;http://purl.bioontology.org/ontology/SNOMEDCT/255621006',
 'http://purl.bioontology.org/ontology/SNOMEDCT/230118003',
 'http://purl.bioontology.org/ontology/SNOMEDCT/255621006',
 'http://purl.bioontology.org/ontology/SNOMEDCT/256317002',
 'http://purl.bioontology.org/ontology/SNOMEDCT/29263009',
 'http://purl.bioontology.org/ontology/SNOMEDCT/29263009;http://purl.bioontology.org/ontology/SNOMEDCT/255621006',
 'http://purl.bioontology.org/ontology/SNOMEDCT/35748005',
 'http://purl.bioontology.org/ontology/SNOMEDCT/412552006;http://purl.bioontology.org/ontology/SNOMEDCT/22836000',
 'http://purl.bioontology.org/ontology/SNOMEDCT/53410008',
 'http://purl.bioontology.org/ontology/SNOMEDCT/70813002',
 'http://purl.bioontology.org/ontology/SNOMEDCT/72511004;http://purl.bioontology.org/ontology/SNOMEDCT/22836000',
 'http://purl.bioontology.org/ontology/SNOMEDCT/735051003',
 'http://purl.bioontology.org/ontology/SNOMEDCT/735215001',
 'http://purl.bioontology.org/ontology/SNOMEDCT/735249009',
 'http://purl.bioontology.org/ontology/SNOMEDCT/762952008'}

In [94]:
snomedct_mappings=show_phrases_per_class(snomedct_tags)

/tmp/ipykernel_2298/2909265927.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hansard_interesting['words']=hansard_interesting['words'].apply(lambda x: x.lower())


http://purl.bioontology.org/ontology/SNOMEDCT/230118003
drink              20
alcoholic drink     4
dtype: int64
http://purl.bioontology.org/ontology/SNOMEDCT/762952008
peanut                  60
peanut oil              28
unrefined peanut oil     4
dtype: int64
http://purl.bioontology.org/ontology/SNOMEDCT/226465004
soft drinks               28
drinks                    16
alcoholic drinks          12
alcohol drinks             8
energy drinks              4
non - alcoholic drinks     4
soda drinks                4
sweetened drinks           4
dtype: int64
http://purl.bioontology.org/ontology/SNOMEDCT/72511004;http://purl.bioontology.org/ontology/SNOMEDCT/22836000
fruit and vegetable    80
dtype: int64
http://purl.bioontology.org/ontology/SNOMEDCT/226064009
vodka    12
dtype: int64
http://purl.bioontology.org/ontology/SNOMEDCT/226474002;http://purl.bioontology.org/ontology/SNOMEDCT/16313001;http://purl.bioontology.org/ontology/SNOMEDCT/70813002
tea with milk    4
dtype: int64
http://p

In [95]:
snomedct_mappings

{'http://purl.bioontology.org/ontology/SNOMEDCT/230118003': 'drink',
 'http://purl.bioontology.org/ontology/SNOMEDCT/762952008': 'peanut',
 'http://purl.bioontology.org/ontology/SNOMEDCT/226465004': 'soft drinks',
 'http://purl.bioontology.org/ontology/SNOMEDCT/72511004;http://purl.bioontology.org/ontology/SNOMEDCT/22836000': 'fruit and vegetable',
 'http://purl.bioontology.org/ontology/SNOMEDCT/226064009': 'vodka',
 'http://purl.bioontology.org/ontology/SNOMEDCT/226474002;http://purl.bioontology.org/ontology/SNOMEDCT/16313001;http://purl.bioontology.org/ontology/SNOMEDCT/70813002': 'tea with milk',
 'http://purl.bioontology.org/ontology/SNOMEDCT/13577000': 'nut',
 'http://purl.bioontology.org/ontology/SNOMEDCT/227411006': 'sage',
 'http://purl.bioontology.org/ontology/SNOMEDCT/70813002': 'milk',
 'http://purl.bioontology.org/ontology/SNOMEDCT/22836000;http://purl.bioontology.org/ontology/SNOMEDCT/255621006': 'vegetable cream',
 'http://purl.bioontology.org/ontology/SNOMEDCT/226470006;